classify will be they will file a claim or not file a claim

1. there are distribution issues with the data; 
will prob have to resample -- apply SMOTE
2. look out for ordinal data


In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE, ADASYN

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


**Data Dictionary**

- pol_eff_dt - auto insurance policy effective date
- gender - gender of driver: F, M
- agecat - driver's age category: 1 (youngest), 2, 3, 4, 5, 6
- credit_score - driver’s credit score(integer): 1-100, 1=poor, 100=excellent
- area - driver's area of residence: A, B, C, D, E, F
- traffic_index - traffic index of driver’s area of residence(integer): 100=country average, >100 means worse traffic conditions than average
- veh_age - age of vehicle(categorical): 1 (youngest), 2, 3, 4
- veh_body - vehicle body, coded as:
    - BUS
    - CONVT = convertible  
    - COUPE   
    - HBACK = hatchback                  
    - HDTOP = hardtop
    - MCARA = motorized caravan
    - MIBUS = minibus
    - PANVN = panel van
    - RDSTR = roadster
    - SEDAN    
    - STNWG = station wagon
    - TRUCK           
    - UTE = utility
- veh_value - vehicle value, in \\$10,000s
- claim_office - office location of claim handling agent: A, B, C, D
- traffic2 - traffic index of driver’s area of residence(integer): 100=country average, >100 means worse traffic conditions than average.  Null values have imputed mean of full dataset
- veh_value2 - vehicle value, in \\$10,000s. Null values have been replaces with the mean of the full dataset
- veh_value3 - vehicle value, in \\$10,000s. Null values have been replaces with the median of the full dataset
- veh_age2 - age of vehicle(categorical): 1 (youngest), 2, 3, 4.  Null values have been replaces with the most frequent category, which is 2
- credit2 - driver’s credit score(integer): 1-100, 1=poor, 100=excellent. Null values have been replaces with the mean of the full dataset
- credit3 - driver’s credit score(integer): 1-100, 1=poor, 100=excellent. Null values have been replaces with the median of the full dataset
- dob - driver's date of birth
- age - Driver's age at policy effective date
- claim_ind - Indicates if the policy had a claim (=1) or not
- veh_body2 - vehicle body, with a collapsed type coding, as described below.  Original coding on left, new coding on right:
    - 'BUS' = 'MIBUS',
    - 'CONVT' = 'HDTOP'
    - 'COUPE' = 'COUPE'
    - 'HBACK' = 'HBACK'
    - 'HDTOP' = 'HDTOP'
    - 'MCARA' = 'MIBUS'
    - 'MIBUS' = 'MIBUS'
    - 'PANVN' = 'PANVN'
    - 'RDSTR' = 'HDTOP'
    - 'SEDAN' = 'SEDAN'
    - STNWG' = 'STNWG'
    - 'TRUCK' = 'TRUCK'
    - 'UTE' = 'UTE'

- numclaims - number of claims(integer): 0 if no claim
- claimcst0 - claim amount: 0 if no claim.  This is the total one year claim amount (ultimate loss) for the policy. 


# Obtaining the Data

In [2]:
data = pd.read_csv('autoinsurance_claims.csv')
data.head()

C:\Users\mngav\Anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,3,4,6,7,9,11,12,13,14,15,16,18,19,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,,,,,,,,,,,,,,,,,,,,,,<<<<<<< HEAD
﻿pol_number,pol_eff_dt,gender,agecat,credit_score,area,traffic_index,veh_age,veh_body,veh_value,claim_office,traffic2,veh_value2,veh_value3,veh_age2,credit2,credit3,dob,age,claim_ind,veh_body2,numclaims,claimcst0
80750205,7/1/2015,M,4,52,C,93,4,STNWG,2.96,NaN,93,2.96,2.96,4,52,52,2/18/1963,53,0,STNWG,0,0
90994433,8/1/2015,M,3,83,C,108,4,SEDAN,1.89,NaN,108,1.89,1.89,4,83,83,2/18/1978,38,0,SEDAN,0,0
59239520,7/1/2015,M,5,71,A,NaN,3,SEDAN,2.75,NaN,100,2.75,2.75,3,71,71,4/14/1960,56,0,SEDAN,0,0
67030771,11/1/2015,M,3,NaN,B,77,3,UTE,2.92,NaN,77,2.92,2.92,3,74.73088342,81,12/23/1974,41,0,UTE,0,0


In [3]:
data.shape

(121900, 1)

# Preprocessing

In [4]:
# looking for null values
data.isnull().sum()

<<<<<<< HEAD    2
dtype: int64

__Dropping Columns__

In [ ]:
# claim office is mostly empty; lets drop the feature
# we don't need the policy number either
# veh_body and veh_body2 are redundant, keep veh_body2 as it is updated
# the dateTime columns create a complexity we could do without.  pol_eff_dt is proprietary info we don't need and dob is
  # redundant since we have age. 
data.drop(['claim_office', 'pol_number', 'veh_body', 'pol_eff_dt', 'dob'], axis = 1, inplace = True)




there are duplicate columns for credit_score, traffic_index, vehicle_value, containing null values, where either the mean, median or both were used to impute missing values.  Lets keep only columns with median imputation

In [ ]:
# drop credit_score and credit2 columns
data.drop(['credit_score', 'credit2', 'traffic_index', 'veh_value', 'veh_value2'], axis = 1, inplace = True)

vehicle age also has a duplicate column, where the mode was used for null values.  This is an ordinal value so keeping the mode for imputation makes sense

In [ ]:
# drop veh_age column
data.drop('veh_age', axis =1, inplace=True)

In [ ]:
# confirm we have no other null values
data.isnull().sum()

**Fix Data Types**

In [ ]:
data.info()



| Feature | Change Type to: |
|------------|---------------------|
| pol_eff_dt | change to date time |
| gender | change to category |
| agecat | change to category |
| area | change to category |
| veh_age2 | change to category |
| dob | change to DateTime |
| claim_ind | change to category |
| veh_body2 | change to category |


In [ ]:
data = data.astype({'gender':'category', 'area':'category', 
             'claim_ind':'category', 'veh_body2':'category'}, inplace = True)

In [ ]:
# double check the work
data.info()

**Edit Column Names**

Lets change a few of the column names for ease of use as we move forward and work with the data

In [ ]:
data.rename(columns={'traffic2':'traffic_index', 'veh_value3':'veh_value', 'credit3':'credit_score', 'numclaims':'num_claims', 
             'claimcst0':'claim_amount', 'veh_age2':'veh_ageIndex'}, inplace=True)

**Look for Multicolinearity**

In [ ]:
plt.figure(figsize=(15, 12))
sns.heatmap(data.corr(), annot=True)

we see some highly correlated independent variables.  Lets get through the vanilla model and then decided what features to drop.

**Split data set between categorical and continuous**

I will treat ordinal features as numeric data.  Doing this requires the assumption that the numeric space between each category is equal or close to equal.   To ignore the ordering would essentially mean we could delete columns for the drivers age, the traffic index, vehicle age and credit score; that would be too much useful information to lose and an extreme downside.  The downside to treating this data as numeric is that if the spread between bins are not equal, then our results may only be approximate.  I would posit that the approximate results would be more accurate than any model that disregarded them. 

In [ ]:
# create continuous sub-DataFrame

continuous = data.copy()
for i in continuous:
    if continuous[i].dtype.name == 'category':
        continuous.drop(i, axis=1, inplace = True)
continuous.info()

In [ ]:
# create categorical sub-DataFrame

categorical = data.copy()
for i in categorical:
    if categorical[i].dtype.name != 'category':
        categorical.drop(i, axis = 1, inplace=True)
categorical.info()

In [ ]:
# inspect distributions for continuous data
continuous.hist(figsize=(10,10))

**Get Dummies for categorical data**

In [ ]:
categorical.head()

In [ ]:
categorical.info()

In [ ]:
categorical.veh_body2.value_counts()

In [ ]:
# one-hot encode categorical data
dummies = pd.get_dummies(categorical.drop('claim_ind', axis=1))
dummies.head()

**concat the continuous and categorical DataFrames**

In [ ]:
df = pd.concat([continuous, dummies, categorical.claim_ind], axis = 1, sort=False)
df.head()

# Logistic Regression Models

Lets check for any class imbalance in our target variable

In [ ]:
df.claim_ind.value_counts()

Roughly 7% of the target data are True while 93% are False so, it looks like we do have class imbalance and will run SMOTE to fix it.

In [ ]:
# define X and y 
X = df[df.columns[:-1]]
y = df.claim_ind

# resample X and y wiht SMOTE 
X_resampled, y_resampled = SMOTE().fit_sample(X, y)

# preview synthtic sample claim_ind distribution
print(pd.Series(y_resampled).value_counts())

In [ ]:
# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.3, random_state = 42)